In [1]:
import numpy as np
import numpy.linalg as la
import scipy.linalg as sla
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import animation,rc
from IPython.display import HTML


# Finite Difference Method 

## 1) $f: R \rightarrow R$


Suppose you have the function:

$$ f(x) = e^x - 2 $$

and we want to use FD difference method to compute $\frac{df}{dx}$. We will first define the functions to evaluate $f(x)$ and $\frac{df}{dx}(x)$ exactly.

In [ ]:
def f(x):
    return np.exp(x) - 2

def df(x):
    return np.exp(x)

In [ ]:
x = np.linspace(-1, 2, 100)
plt.figure(figsize=(6,6))
plt.plot(x, f(x), lw=3)

Let's evaluate the first derivative using finite difference approximation for decreasing values of h. In this example, we want to approximate the derivative at $x = \hat{x} = 1.0$.

In [ ]:
xhat = 1.0  # point where we want to find the approximation
h = 1.0     # initial perturbation
errors = []
hs = []

fval = f(xhat)  # we only need to evaluate this once!

# in general, we don't have this value, but we will use it here to visualize the error
dfexact = df(xhat)  

for i in range(20):
    
    # one function evaluation per each perturbation size
    dfapprox = ( f(xhat+h) - fval ) / h   
    
    # get the error
    err = np.abs(dfexact - dfapprox)
    
    print(" %E \t %E " %(h, err) )
    
    hs.append(h)
    errors.append(err)
    
    h = h / 2

In [ ]:
plt.figure(figsize=(16,10))
plt.loglog(hs, errors, lw=3)
plt.xlabel('h')
plt.ylabel('error')
plt.xlim([1e-6,1])
plt.ylim([1e-6,1])
plt.grid()

What happens if we keep decreasing the perturbation h?

In [ ]:
xhat = 1.0  # point where we want to find the approximation
h = 1.0     # initial perturbation
errors = []
hs = []

fval = f(xhat)  # we only need to evaluate this once!

# in general, we don't have this value, but we will use it here to visualize the error
dfexact = df(xhat)  

for i in range(80):
    
    # one function evaluation per each perturbation size
    dfapprox = ( f(xhat+h) - fval ) / h   
    
    # get the error
    err = np.abs(dfexact - dfapprox)
    
    print(" %E \t %E " %(h, err) )
    
    hs.append(h)
    errors.append(err)
    
    h = h / 2

In [ ]:
plt.figure(figsize=(16,10))
plt.loglog(hs, errors, lw=3)
plt.xlabel('h')
plt.ylabel('error')
plt.grid()

Not what we expected, correct? If we plot the asymptotic behavior of the truncation error, the error continues to become smaller as the perturbation size decreases:

In [ ]:
plt.figure(figsize=(16,10))
plt.loglog(hs, errors, lw=3,label='total')
plt.loglog(hs,np.array(hs),'--',label='truncation')
plt.loglog(hs,1e-16/np.array(hs),'--',label='rounding')
plt.legend(bbox_to_anchor=(1.1, 1.05))
plt.xlabel('h')
plt.ylabel('error')
plt.grid()
plt.ylim(1e-16,1e2)

However, as the perturbation decreases, cancelation starts to take effect when computing the finite difference. There is a competition between the truncation error and the rounding error.

## 2) $f: R^n \rightarrow R$

Suppose you have the function:

$$ f(x_1, x_2) = 2 x_1 + x_1^2 \, x_2 + x_2^3 $$

and we want to use FD difference method to compute $\frac{\partial f}{\partial x_1}$ and $\frac{\partial f}{\partial x_2}$

In [ ]:
def f(x1,x2):
    return (2*x1 + (x1**2)*x2 + x2**3)

You will need to add a perturbation to each one of the function variables $x_i$. The "ideal" perturbation is not necessarily the same for each variable, but it can be a good starting point. We will perform these computations for $x_1 = 1.3$ and $x_2 = 4.9$, using the Forward Finite Difference Method. The exact solution is:

In [ ]:
x1 = 1.3
x2 = 4.9
dfdx1_exact = 2 + 2*x1*x2
dfdx2_exact = x1**2 + 3*x2**2
print(dfdx1_exact,dfdx2_exact)

In [ ]:
h = 0.05
fval = f(x1,x2)

dfx1 = (f(x1+h,x2)-fval)/h
dfx2 = (f(x1,x2+h)-fval)/h

print(dfx1, dfdx1_exact, abs((dfx1-dfdx1_exact)/dfdx1_exact) )
print(dfx2, dfdx2_exact, abs((dfx2-dfdx2_exact)/dfdx2_exact) )

Note that in this case, you have as many derivatives as you have variables. Indeed, your function derivative follows:


$$f': R^n \rightarrow R^n $$

You can think of storing these quantities as an array. Supposed the function is:

$$ f({\bf x}) = f(x_1, x_2, x_3) = 2 x_1 x_3 + x_1^2 \, x_2 + x_2^3 + 6 x_3^2$$

But we will define it as:

In [ ]:
def f(xvec):
    return (2*xvec[0]*xvec[2] + (xvec[0]**2)*xvec[1] + xvec[1]**3 + 6*xvec[2]**2)

In general, for a function $f(\bf x)$, the derivative can be expressed as:

    
$$ \nabla f = \begin{bmatrix} \frac{\partial f}{\partial x_1}\\\frac{\partial f}{\partial x_2}\\...\\ \frac{\partial f}{\partial x_n} \end{bmatrix} $$

which is called the **gradient** of $f$. The exact array can be obtained using:

In [ ]:
def gradf(xvec):
    x1,x2,x3 = xvec
    df1 = 2*x3 + 2*x1*x2
    df2 = x1**2 + 3*x2**2
    df3 = 2*x1 + 12*x3
    return np.array([df1,df2,df3])

We will perform our computation using:

In [ ]:
xvec = np.array([4.1, 3.3, 15.8])

Remember that we need to add the perturbation one entry at a time! We should also compute the function value for the unperturbed variables only once.

In [ ]:
fval = f(xvec)
print(fval)

In [ ]:
h = 0.1
df_grad = np.zeros_like(xvec)
for i in range(len(xvec)):
    xfd = xvec.copy()
    xfd[i] += h
    df_grad[i] = (f(xfd) - fval)/h

print(df_grad)
print(gradf(xvec))

## 2) $f: R^n \rightarrow R^m$

Suppose you have the function ${\bf f}({\bf x})$ where ${\bf x} \in R^n$:

$$ {\bf f}({\bf x}) =  \begin{bmatrix} f_1({\bf x})\\f_2({\bf x})\\ ... \\f_m({\bf x}) \end{bmatrix} $$

The derivative of this function is:

$$ {\bf J}({\bf x}) =  \begin{bmatrix} 
\frac{\partial f_1}{\partial x_1}({\bf x}) & ... & \frac{\partial f_1}{\partial x_i}({\bf x}) & ... & \frac{\partial f_1}{\partial x_n}({\bf x})\\
 & & \ddots & &\\
\frac{\partial f_m}{\partial x_1}({\bf x}) & ... & \frac{\partial f_m}{\partial x_i}({\bf x}) & ... & \frac{\partial f_m}{\partial x_n}({\bf x})\\
\end{bmatrix}  =
\begin{bmatrix} 
\frac{\partial {\bf f}}{\partial x_1} & ... & \frac{\partial {\bf f}}{\partial x_i} & ... & \frac{\partial {\bf f}}{\partial x_n}
\end{bmatrix} $$

which is known as the **Jacobian** matrix.

Here is another example:

In [ ]:
def fn(xvec):
    f1 = 2*xvec[0]*xvec[2] + (xvec[0]**2)*xvec[1] + xvec[1]**3 + 6*xvec[2]**2
    f2 = xvec[0]*xvec[1]*xvec[2]   
    return np.array([f1,f2])

In [ ]:
xvec = np.array([42.1, 33.3, 15.8])

In [ ]:
fn(xvec)

The exact Jacobian is:

In [ ]:
def Jacobianf(xvec):
    x1,x2,x3 = xvec
    df1dx1 = 2*x3 + 2*x1*x2
    df1dx2 = x1**2 + 3*x2**2
    df1dx3 = 2*x1 + 12*x3
    df2dx1 = x2*x3
    df2dx2 = x1*x3
    df2dx3 = x2*x1
    return np.array([[df1dx1,df1dx2,df1dx3],[df2dx1,df2dx2,df2dx3]])   

In [ ]:
Jacobianf(xvec)

Using finite difference method:

In [ ]:
h = 0.001
m = 2
n = 3
Jac = np.zeros((m,n))
fvaln = fn(xvec)

for i in range(n):
    xfd = xvec.copy()
    xfd[i] += h
    Jac[:,i] = (fn(xfd) - fvaln)/h

print(Jac)
print(Jacobianf(xvec))

# Diffusion Equation Finite Difference equation


<div class="alert alert-info">

<h1>Motivation: Cooling of a rod given an initial temperature </h1>

The time-dependent diffusion equation is a partial differential equation (PDE) that is often used to model heat transfer problems:
    
$$\frac{\partial u}{\partial t} - D\frac{\partial^2 u}{\partial x^2} = 0$$
    
where $u$ is the temperature that varies in space $x$ and time $t$, i.e., $u(x,t)$. $D$ is the thermal diffusivity.
    
Boundary conditions: $u(-1,t) = 0 \textrm{ and } u(1,t) = 0 \textrm{   for any    } t > 0$

Initial conditions: $u(x,0) = u_0 = 0.5 \textrm{   for any    } x \in [-1,1]$
    

    
In a previous notebook, you learned how to solve this problem using the SEM method. We will now solve this same example using the **Finite Difference Method**.
    
</div>

We will use the Backward Finite Difference to approximate $\frac{\partial u}{\partial t}$:
    
    
$$ \frac{\partial u}{\partial t} = \frac{u(x_i, t^{(j)}) - u(x_i, t^{(j-1)}) }{\Delta t} $$

where we will consider equal time increments ${\Delta t}  = t^{(j)} - t^{(j-1)}$.


And we will use the Second order Central Finite Difference to approximate $\frac{\partial^2 u}{\partial x^2} $:
    
    
$$ \frac{\partial^2 u}{\partial x^2}  = \frac{u(x_{i+1}, t^{(j)}) - 2 \, u(x_i, t^{(j)}) + u(x_{i-1}, t^{(j)}) }{\Delta x} $$

where we will consider equal spacing increments ${\Delta x}  = x_{i+1} - x_i$.

Plugging these approximations into the PDE, we get the following:


$$ \frac{D \Delta t}{\Delta x} \left( - u(x_{i+1}, t^{(j)}) + 2 \, u(x_i, t^{(j)}) - u(x_{i-1}, t^{(j)}) \right) +  u(x_i, t^{(j)}) = u(x_i, t^{(j-1)}) $$


which we can solve using the a time-stepping scheme as before: $\mathbf{A}\,\mathbf{u}^{(j)} = \mathbf{u}^{(j-1)}$ where 

$$ \mathbf{u}^{(j)} = \begin{bmatrix}  u(x_0, t^{(j)}) \\ u(x_1, t^{(j)}) \\ ... \\ u(x_{N-1}, t^{(j)}) \\ u(x_N, t^{(j)}) \end{bmatrix} = \begin{bmatrix}  0 \\ u(x_1, t^{(j)}) \\ ... \\ u(x_{N-1}, t^{(j)})  \\ 0 \end{bmatrix} $$


Since we are only solving for the $N-1$ "internal" points, we need to construct a matrix with dimensions $N-1 \times N-1$. We will use the following parameters:

In [2]:
N = 50                      # number of spacing (intervals)
x = np.linspace(-1,1,N+1)   # these are the discretized points (and so we have N + 1 points)
dx = x[1] - x[0]

D = 0.1              # thermal diffusivity
dt = 0.05            # time increment
timesteps = 400      # number of timesteps
u0 = 0.5             # initial temperature in the rod

utotal = np.zeros((x.shape[0],timesteps+1))  # to store all the temperatures

Write the initial vector  $\mathbf{u}({\bf x},t^0) = \mathbf{u}_0$ (initial condition)

In [3]:
ut0 = u0*np.ones(N+1)

Update the first column of `utotal` using `ut0`

In [4]:
utotal[:,0] = ut0

Build the FD matrix ${\bf A}$. 

In [5]:
B = np.diag(2*np.ones(N-1)) - np.diag(np.ones(N-2),1) - np.diag(np.ones(N-2),-1)
A = np.eye(N-1) + D*(dt/(dx**2))*B

Use `plt.spy` to see structure of the matrix

In [6]:
plt.spy(A,markersize = 1)

The matrix ${\bf A}$ is called a `banded matrix`, because of the distribution of the non-zero entries. There are special solvers that take into consideration this type of storage, making the overall computational a lot more efficient (for example `scipy.linalg.solve_banded`). 

For the sake of simplicity, we will just use a "general solve" here.

In [7]:
for i in range(timesteps):
    u_old = utotal[:,i]
    b = u_old[1:-1]  # remove the boundary conditions
    u_new = sla.solve(A,b)
    utotal[:,i+1] = np.append(0,np.append(u_new,0)) # add back the boundary condition

In [8]:
fig,ax = plt.subplots()
ax.set_xlim((-1,1))
ax.set_ylim((0,1.0))
line, = ax.plot([],[],lw=2)

def init():
    line.set_data([],[])
    return (line,)

def animate(i):
    y = utotal[:,i]
    line.set_data(x,y)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=50, blit=True)

rc('animation',html='html5')
anim